In [25]:
import duckdb
import pandas as pd
import os
from datetime import datetime

In [26]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [40]:
df = pd.read_csv('landing\z0019_2.csv', sep=';', encoding='latin1', low_memory=False)

In [41]:
df.head()

,MATBR,MAKTX,WERKS,MAINS,LABST
0,10004,SERRA,BT50,100,200
1,10005,MACHADO,BT50,100,100
2,10003,PREGO,BT10,100,60


In [42]:
data_ingestao = datetime.now()

In [43]:
arquivo = 'landing\z0019_2.csv'
data_ingestao = datetime.now()
df['nome_arquivo']=arquivo
df['data_ingestao']=data_ingestao
df.head()

,MATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao
0,10004,SERRA,BT50,100,200,landing\z0019_2.csv,2025-03-16 20:50:10.095225
1,10005,MACHADO,BT50,100,100,landing\z0019_2.csv,2025-03-16 20:50:10.095225
2,10003,PREGO,BT10,100,60,landing\z0019_2.csv,2025-03-16 20:50:10.095225


In [44]:
con.execute("""
    CREATE TABLE IF NOT EXISTS bronze_produtos (
        MATBR VARCHAR,
        MATKL VARCHAR,
        WGBEZ VARCHAR,
        MAINS VARCHAR,
        LABST VARCHAR,
        nome_arquivo VARCHAR,
        data_ingestao TIMESTAMP
    )               
""")

In [47]:
con.execute("INSERT INTO bronze_produtos SELECT * FROM df")

In [49]:
resultado = con.execute("SELECT * FROM bronze_produtos").fetchdf()
resultado.head(6)

,MATBR,MATKL,WGBEZ,MAINS,LABST,nome_arquivo,data_ingestao
0,10001,PARAFUSO,BT10,100,100,landing\z0019_1.csv,2025-03-16 20:38:30.340301
1,10002,MARTELO,BT50,100,1500,landing\z0019_1.csv,2025-03-16 20:38:30.340301
2,10003,PREGO,BT10,100,50,landing\z0019_1.csv,2025-03-16 20:38:30.340301
3,10004,SERRA,BT50,100,200,landing\z0019_2.csv,2025-03-16 20:50:10.095225
4,10005,MACHADO,BT50,100,100,landing\z0019_2.csv,2025-03-16 20:50:10.095225
5,10003,PREGO,BT10,100,60,landing\z0019_2.csv,2025-03-16 20:50:10.095225


In [50]:
con.execute("ALTER TABLE bronze_produtos RENAME TO bronze_z0019")

In [51]:
resultado = con.execute("SELECT * FROM bronze_z0019").fetchdf()
resultado.head(6)

,MATBR,MATKL,WGBEZ,MAINS,LABST,nome_arquivo,data_ingestao
0,10001,PARAFUSO,BT10,100,100,landing\z0019_1.csv,2025-03-16 20:38:30.340301
1,10002,MARTELO,BT50,100,1500,landing\z0019_1.csv,2025-03-16 20:38:30.340301
2,10003,PREGO,BT10,100,50,landing\z0019_1.csv,2025-03-16 20:38:30.340301
3,10004,SERRA,BT50,100,200,landing\z0019_2.csv,2025-03-16 20:50:10.095225
4,10005,MACHADO,BT50,100,100,landing\z0019_2.csv,2025-03-16 20:50:10.095225
5,10003,PREGO,BT10,100,60,landing\z0019_2.csv,2025-03-16 20:50:10.095225


In [52]:
con.close()

# Camada Silver

In [53]:
import duckdb

In [54]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [55]:
df = con.execute("SELECT * FROM bronze_z0019").fetchdf()
df.head(10)

,MATBR,MATKL,WGBEZ,MAINS,LABST,nome_arquivo,data_ingestao
0,10001,PARAFUSO,BT10,100,100,landing\z0019_1.csv,2025-03-16 20:38:30.340301
1,10002,MARTELO,BT50,100,1500,landing\z0019_1.csv,2025-03-16 20:38:30.340301
2,10003,PREGO,BT10,100,50,landing\z0019_1.csv,2025-03-16 20:38:30.340301
3,10004,SERRA,BT50,100,200,landing\z0019_2.csv,2025-03-16 20:50:10.095225
4,10005,MACHADO,BT50,100,100,landing\z0019_2.csv,2025-03-16 20:50:10.095225
5,10003,PREGO,BT10,100,60,landing\z0019_2.csv,2025-03-16 20:50:10.095225
6,10004,SERRA,BT50,100,200,landing\z0019_2.csv,2025-03-16 20:50:10.095225
7,10005,MACHADO,BT50,100,100,landing\z0019_2.csv,2025-03-16 20:50:10.095225
8,10003,PREGO,BT10,100,60,landing\z0019_2.csv,2025-03-16 20:50:10.095225


In [59]:
df = con.execute("""
                SELECT *
                FROM (
                    SELECT *, ROW_NUMBER() OVER (PARTITION BY MATBR ORDER BY data_ingestao DESC) AS row
                    FROM bronze_z0019
                    WHERE data_ingestao >= '2025-01-01'
                 ) WHERE row = 1
                   """).fetchdf()
df.head(10)

,MATBR,MATKL,WGBEZ,MAINS,LABST,nome_arquivo,data_ingestao,row
0,10003,PREGO,BT10,100,60,landing\z0019_2.csv,2025-03-16 20:50:10.095225,1
1,10001,PARAFUSO,BT10,100,100,landing\z0019_1.csv,2025-03-16 20:38:30.340301,1
2,10004,SERRA,BT50,100,200,landing\z0019_2.csv,2025-03-16 20:50:10.095225,1
3,10002,MARTELO,BT50,100,1500,landing\z0019_1.csv,2025-03-16 20:38:30.340301,1
4,10005,MACHADO,BT50,100,100,landing\z0019_2.csv,2025-03-16 20:50:10.095225,1


In [60]:
df.head(10)

,MATBR,MATKL,WGBEZ,MAINS,LABST,nome_arquivo,data_ingestao,row
0,10003,PREGO,BT10,100,60,landing\z0019_2.csv,2025-03-16 20:50:10.095225,1
1,10001,PARAFUSO,BT10,100,100,landing\z0019_1.csv,2025-03-16 20:38:30.340301,1
2,10004,SERRA,BT50,100,200,landing\z0019_2.csv,2025-03-16 20:50:10.095225,1
3,10002,MARTELO,BT50,100,1500,landing\z0019_1.csv,2025-03-16 20:38:30.340301,1
4,10005,MACHADO,BT50,100,100,landing\z0019_2.csv,2025-03-16 20:50:10.095225,1


In [64]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'row'])
df_final.head(10)

,MATBR,MATKL,WGBEZ,MAINS,LABST
0,10003,PREGO,BT10,100,60
1,10001,PARAFUSO,BT10,100,100
2,10004,SERRA,BT50,100,200
3,10002,MARTELO,BT50,100,1500
4,10005,MACHADO,BT50,100,100


In [65]:
df_final = df_final.rename(columns={'MATBR':'id'})
df_final = df_final.rename(columns={'MATKL':'nome_produto'})
df_final = df_final.rename(columns={'WGBEZ':'id_categoria'})
df_final = df_final.rename(columns={'MAINS':'id_fornecedor'})
df_final = df_final.rename(columns={'LABST':'preco'})
df_final.head(10)

,id,nome_produto,id_categoria,id_fornecedor,preco
0,10003,PREGO,BT10,100,60
1,10001,PARAFUSO,BT10,100,100
2,10004,SERRA,BT50,100,200
3,10002,MARTELO,BT50,100,1500
4,10005,MACHADO,BT50,100,100


# Camada Gold

In [69]:
df2 = df_final
df2 = df2.astype(
    {
        'id': 'int',
        'nome_produto': 'str',
        'id_categoria': 'str',
        'id_fornecedor': 'int',
        'preco': 'float'
    }
)
df2.head(10)

,id,nome_produto,id_categoria,id_fornecedor,preco
0,10003,PREGO,BT10,100,60.0
1,10001,PARAFUSO,BT10,100,100.0
2,10004,SERRA,BT50,100,200.0
3,10002,MARTELO,BT50,100,1500.0
4,10005,MACHADO,BT50,100,100.0


In [70]:
df2.dtypes


id                 int32
nome_produto      object
id_categoria      object
id_fornecedor      int32
preco            float64
dtype: object

In [71]:
con.execute("""
CREATE TABLE IF NOT EXISTS produtos (
            id BIGINT,
            nome_produto TEXT,
            id_categoria TEXT,
            id_fornecedor BIGINT,
            preco FLOAT
            )
""")

In [73]:
con.execute("INSERT INTO produtos SELECT * FROM df2")

In [74]:
df_resultado = con.execute("SELECT * FROM produtos").fetchdf()
df_resultado.head(10)

,id,nome_produto,id_categoria,id_fornecedor,preco
0,10003,PREGO,BT10,100,60.0
1,10001,PARAFUSO,BT10,100,100.0
2,10004,SERRA,BT50,100,200.0
3,10002,MARTELO,BT50,100,1500.0
4,10005,MACHADO,BT50,100,100.0


In [75]:
#con.close()

# Dimensão na Gold e Pull Request

In [77]:
df = con.execute("SELECT * FROM produtos").fetchdf()
df.head(10)

,id,nome_produto,id_categoria,id_fornecedor,preco
0,10003,PREGO,BT10,100,60.0
1,10001,PARAFUSO,BT10,100,100.0
2,10004,SERRA,BT50,100,200.0
3,10002,MARTELO,BT50,100,1500.0
4,10005,MACHADO,BT50,100,100.0


In [78]:
df2 = df.drop(columns=['id_categoria', 'id_fornecedor'])
df2.head(10)

,id,nome_produto,preco
0,10003,PREGO,60.0
1,10001,PARAFUSO,100.0
2,10004,SERRA,200.0
3,10002,MARTELO,1500.0
4,10005,MACHADO,100.0


In [80]:
con.execute(""" 
CREATE TABLE IF NOT EXISTS dim_produtos (
            id BIGINT,
            nome_produto TEXT,
            preco FLOAT
            )
""")

In [83]:
df_dim = con.execute("SELECT * FROM dim_produtos").fetchdf()
df_dim.head(10)

,id,nome_produto,preco
0,10003,PREGO,60.0
1,10001,PARAFUSO,100.0
2,10004,SERRA,200.0
3,10002,MARTELO,1500.0
4,10005,MACHADO,100.0


In [82]:
con.execute("INSERT INTO dim_produtos SELECT * FROM df2")

In [84]:
con.close()